In [17]:
import json
import pandas as pd

In [22]:
# Read in the geojson file
data = None
with open("berney_divisions.geojson", "r") as f:
    data = json.load(f)

# Read website\public\data\bipartite_divisions_type_metiers.csv
df = pd.read_csv("bipartite_divisions_type_metiers.csv")

# Read data/1832_v4_preprocessed.csv
df_pop = pd.read_csv("../../../data/1832_v4_preprocessed.csv")

# Get the population of each division
population = df_pop.groupby("division").size().to_dict()

# Get the origin category for eachn division
origin = df_pop.groupby(["division", "chef_origine_class"]).count()["nom_rue"]

# In the properties section, add the values
# For example, for division 1:
# "properties": {"id": 1, "name": "marterey", "jobs": {"agricole": 82, "artisanat": 93,...
for feature in data["features"]:
    division = feature["properties"]["id"]
    # Add the population
    feature["properties"]["population"] = population[division]
    division_df = df.query("division == @division")
    jobs = {}
    for index, row in division_df.iterrows():
        jobs[row["job_category"]] = row["Weight"]
    feature["properties"]["jobs"] = jobs

# In the properties section, add the origins values from the origin dataframe
# For example, for division 1:
# "properties": {"id": 1, "name": "marterey", "origins": {"aigle": 1, moudon: 6, ...}
for feature in data["features"]:
    division = feature["properties"]["id"]
    origins = {}
    for index, row in origin[division].items():
        origins[index] = row
    # Take the sum of all origins outside Lausanne from the origin dataframe
    origins["not_lausanne"] = int(origin[division].sum() - origin[division]["lausanne"])
    feature["properties"]["origins"] = origins

In [23]:
df.job_category.unique()

array(['administration', 'agricole', 'artisanat', 'commerce',
       'construction', 'rente', 'service'], dtype=object)

In [ ]:
# Add a new origin category "not_lausanne" to the origin dataframe
df_pop["chef_origine_class"] = df_pop["chef_origine_class"].fillna("not_lausanne")

In [26]:
# Replace all origins that are not Lausanne with "not_lausanne"
df_pop["chef_origine_class"] = df_pop["chef_origine_class"].apply(lambda x: "not_lausanne" if x != "lausanne" else x)

df_pop.groupby(["division", "chef_origine_class"]).count()["nom_rue"].reset_index().groupby("chef_origine_class").max()["nom_rue"]

chef_origine_class
lausanne        149
not_lausanne    370
Name: nom_rue, dtype: int64

In [25]:
# Save the new geojson file
with open("berney_divisions.geojson", "w") as f:
    json.dump(data, f)